In [34]:
import pandas as pd
import tensorflow as tf

In [2]:
!pip install transformers beautifulsoup4

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [3]:
tokenizer= AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model= AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\DIBYOJIT\AppData\Local\RStudio\ctx\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
tokens= tokenizer.encode('Wow! Its a lovely experience. I have never been exposed to something like that before', return_tensors='pt')

In [9]:
tokens

tensor([[  101, 94608,   106, 10491,   143, 83944, 16277,   119,   151, 10574,
         13362, 10662, 47617, 10114, 19501, 11531, 10203, 11364,   102]])

In [10]:
tokenizer.decode(tokens[0])

'[CLS] wow! its a lovely experience. i have never been exposed to something like that before [SEP]'

In [11]:
results= model(tokens)

In [14]:
results

SequenceClassifierOutput(loss=None, logits=tensor([[-2.6392, -2.8140, -1.1300,  1.7278,  3.8367]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [17]:
torch.argmax(results.logits)

tensor(4)

In [18]:
int(torch.argmax(results.logits))+1

5

In [19]:
enc= tokenizer.encode('I absolutely hate it. This is totally worthless', return_tensors='pt')

In [20]:
enc

tensor([[  101,   151, 35925, 10563, 39487, 10197,   119, 10372, 10127, 79053,
         25840, 15470,   102]])

In [21]:
encoder= model(enc)

In [22]:
encoder

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.1554,  1.2196, -1.1190, -2.7853, -1.6698]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [24]:
int(torch.argmax(encoder.logits))+1

1

In [26]:
r= requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup= BeautifulSoup(r.text, 'html.parser')
exp= re.compile('.*comment.')
results= soup.find_all('p', {'class':exp})
review= [comment.text for comment in results]

In [29]:
results

[<p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.</span></p>,
 <p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick 

In [30]:
review

["Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future.",
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to che

In [32]:
results[0].text

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

In [39]:
import numpy as np

df= pd.DataFrame(np.array(review), columns=['reviews'])

In [40]:
df.head()

,reviews
0,Seated without a booking on a super busy Satur...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...


In [42]:
df['reviews'][0]

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

In [43]:
def senti_ana(reviews):
    tokenizer= AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    model= AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    tokens= tokenizer.encode(reviews, return_tensors='pt')
    result= model(tokens)
    return int(torch.argmax(result.logits))+1

In [44]:
senti_ana(df['reviews'][0])

C:\Users\DIBYOJIT\AppData\Local\RStudio\ctx\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


5

In [45]:
senti_ana(df['reviews'][1])

2

In [46]:
df['sentiment']= df['reviews'].apply(lambda x: senti_ana(x))

C:\Users\DIBYOJIT\AppData\Local\RStudio\ctx\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [48]:
df

,reviews,sentiment
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4


In [50]:
r2= requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup= BeautifulSoup(r2.text, 'html.parser')
exp= re.compile('.*comment.')
results= soup.find_all('p', {'class':exp})
review= [comment.text for comment in results]

In [51]:
results

[<p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. <br/><br/>The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. <br/><br/>I definitely enjoyed this cafe, the outdoor seating, the service and the food!!</span></p>,
 <p class="comment__09f24__D0cxf y-css-h9c2fl"><span class="raw__09f24__T4Ezm" lang="en">Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. <br/>We all enjoyed our food, as well as our coffe

In [52]:
review

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

In [54]:
df2= pd.DataFrame(review, columns=['Opinions'])

In [55]:
df2.head()

,Opinions
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,We came for brunch and they ran out of seven s...
3,Great place with delicious food and friendly s...
4,The food was delicious. The ricotta pancakes w...


In [56]:
senti_ana(df2['Opinions'][0])

C:\Users\DIBYOJIT\AppData\Local\RStudio\ctx\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


5